Implement the in-place heap-sort algorithm. Experimentally compare its running time with that of the standard heap-sort that is not in-place.

In [ ]:
import heapq
import time
import random

def in_place_heap_sort(arr):
    def heapify(arr, n, i):
        largest = i
        left = 2 * i + 1
        right = 2 * i + 2

        if left < n and arr[left] > arr[largest]:
            largest = left

        if right < n and arr[right] > arr[largest]:
            largest = right

        if largest != i:
            arr[i], arr[largest] = arr[largest], arr[i]
            heapify(arr, n, largest)

    n = len(arr)

    for i in range(n // 2 - 1, -1, -1):
        heapify(arr, n, i)

    for i in range(n - 1, 0, -1):
        arr[0], arr[i] = arr[i], arr[0]
        heapify(arr, i, 0)


def standard_heap_sort(arr):
    heap = []
    for element in arr:
        heapq.heappush(heap, element)

    sorted_arr = []
    while heap:
        sorted_arr.append(heapq.heappop(heap))

    return sorted_arr


def compare_sorting_algorithms():
    sizes = [10_000, 50_000, 100_000]
    results = {}

    for size in sizes:
        test_data = [random.randint(0, 1_000_000) for _ in range(size)]

        in_place_data = test_data.copy()
        start_time = time.time()
        in_place_heap_sort(in_place_data)
        in_place_time = time.time() - start_time

        standard_data = test_data.copy()
        start_time = time.time()
        standard_sorted = standard_heap_sort(standard_data)
        standard_time = time.time() - start_time

        assert in_place_data == sorted(test_data), "In-place heap sort failed"
        assert standard_sorted == sorted(test_data), "Standard heap sort failed"

        results[size] = {
            "In-Place Heap Sort (seconds)": in_place_time,
            "Standard Heap Sort (seconds)": standard_time,
        }

    print("Comparison of Sorting Algorithms:")
    for size, times in results.items():
        print(f"Data Size: {size}")
        for method, time_taken in times.items():
            print(f"  {method}: {time_taken:.6f} seconds")
        print()


if __name__ == "__main__":
    compare_sorting_algorithms()

Comparison of Sorting Algorithms:
Data Size: 10000
  In-Place Heap Sort (seconds): 0.257088 seconds
  Standard Heap Sort (seconds): 0.021866 seconds

Data Size: 50000
  In-Place Heap Sort (seconds): 2.048807 seconds
  Standard Heap Sort (seconds): 0.242338 seconds

Data Size: 100000
  In-Place Heap Sort (seconds): 5.809693 seconds
  Standard Heap Sort (seconds): 0.290496 seconds



P-9.55 Write a program that can process a sequence of stock buy and sell orders
as described in Exercise C-9.50. (An online computer system for trading stocks needs to process orders of the form “buy 100 shares at x dollars each” or “sell 100 shares at y dollars each.” A buy order for x dollars can only be processed if there is an existing sell order with price y dollars such that y ≤ x. Likewise, a sell order for y dollars can only be processed if there is an existing buy order with price x dollars such that y ≤ x.If a buy or sell order is entered but cannot be processed, it must wait for a future order that allows it to be processed. Describe a scheme that allows buy and sell orders to be entered in O(logn) time, independent of whether
or not they can be immediately processed).

In [5]:
import heapq
import time

class StockTrading:
    def __init__(self):
        self.sell_orders = []
        self.buy_orders = []
        self.quantity = 100

    def process_order(self, order_type, price):
        start = time.time()
        if order_type == 'buy':
            self.process_buy(self.quantity, price)
        elif order_type == 'sell':
            self.process_sell(self.quantity, price)
        end = time.time()
        order_time = round(end-start, 8)
        print(f"Order processing time: {order_time} seconds")

    def process_buy(self, quantity, x):
        while self.sell_orders and self.sell_orders[0][0] <= x:
            sell_price, sell_quantity = heapq.heappop(self.sell_orders)
            if sell_quantity > quantity:
                heapq.heappush(self.sell_orders, (sell_price, sell_quantity - quantity))
                print(f"Order processed! You were able to buy your {quantity} shares for {sell_price} dollars each.")
                return
            else:
                print(f"Order processed! You were able to buy your {sell_quantity} shares for {sell_price} dollars each.")
                quantity -= sell_quantity

        if quantity > 0:
            heapq.heappush(self.buy_orders, (-x, quantity))
            print(f"Buy order for {quantity} shares at {x} dollars each added to waiting list.")

    def process_sell(self, quantity, y):  # y is the price for sell orders
        while self.buy_orders and -self.buy_orders[0][0] >= y:
            buy_price, buy_quantity = heapq.heappop(self.buy_orders)
            buy_price = -buy_price
            if buy_quantity > quantity:
                heapq.heappush(self.buy_orders, (-buy_price, buy_quantity - quantity))
                print(f"Order processed! You were able to sell your {quantity} shares for {buy_price} dollars each.")
                return
            else:
                print(f"Order processed! You were able to sell your {buy_quantity} shares for {buy_price} dollars each.")
                quantity -= buy_quantity

        if quantity > 0:
            heapq.heappush(self.sell_orders, (y, quantity))
            print(f"Sell order for {quantity} shares at {y} dollars each added to list.")

    def show_current_orders(self):
        print("\nCurrent Buy Orders (price, quantity):")
        if self.buy_orders:
            for order in self.buy_orders:
                print(f"Buy {abs(order[0])} at {order[1]} shares")
        else:
            print("No pending buy orders.")

        print("\nCurrent Sell Orders (price, quantity):")
        if self.sell_orders:
            for order in self.sell_orders:
                print(f"Sell {order[0]} at {order[1]} shares")
        else:
            print("No pending sell orders.")

def main():
    system = StockTrading()

    while True:
        print("\nWelcome to Stock Trading System")
        print("Buy and Sell your shares")
        print("1. Buy Shares")
        print("2. Sell Shares")
        print("3. Show Current Orders")
        print("4. Exit")

        choice = input("What would you like: ").strip()

        if choice == "1":
            try:
                x = float(input("Enter price you would like to buy 100 shares for: "))
                system.process_order('buy', x)
            except ValueError:
                print("Invalid input. Please enter a number from 1 to 4.")
        elif choice == "2":
            try:
                y = float(input("Enter price you would like to sell 100 shares for: "))
                system.process_order('sell', y)
            except ValueError:
                print("Invalid input. Please enter a number from 1 to 4.")
        elif choice == "3":
            system.show_current_orders()
        elif choice == "4":
            print("Exiting the system.")
            break
        else:
            print("Invalid input. Please enter a number from 1 to 4.")

if __name__ == "__main__":
    main()



Welcome to Stock Trading System
Buy and Sell your shares
1. Buy Shares
2. Sell Shares
3. Show Current Orders
4. Exit
What would you like: 1
Enter price you would like to buy 100 shares for: 20
Buy order for 100 shares at 20.0 dollars each added to waiting list.
Order processing time: 0.00048423 seconds

Welcome to Stock Trading System
Buy and Sell your shares
1. Buy Shares
2. Sell Shares
3. Show Current Orders
4. Exit
What would you like: 2
Enter price you would like to sell 100 shares for: 300
Sell order for 100 shares at 300.0 dollars each added to list.
Order processing time: 0.00014472 seconds

Welcome to Stock Trading System
Buy and Sell your shares
1. Buy Shares
2. Sell Shares
3. Show Current Orders
4. Exit
What would you like: 1
Enter price you would like to buy 100 shares for: 400
Order processed! You were able to buy your 100 shares for 300.0 dollars each.
Order processing time: 0.00013924 seconds

Welcome to Stock Trading System
Buy and Sell your shares
1. Buy Shares
2. Sell